# Quick exercise 14: Notebook adapted from Ryan Abernathey (http://rabernat.github.io/intro_to_physical_oceanography/vorticity_sverdrup_transport_and_gyres.html)

Look at each of the plots generated by the notebook and write down a description of what is shown and what it means

In [ ]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
scow = xr.open_dataset('http://chapman.ceoas.oregonstate.edu/thredds/dodsC/CIOSS/SCOW/Monthly/scow_monthly_maps.nc?latitude[0:1:559],longitude[0:1:1439],time[0:1:11],scow_wind_stress_curl[0:1:11][0:1:1439][0:1:559],scow_zonal_wind_stress[0:1:11][0:1:1439][0:1:559],scow_meridional_wind_stress[0:1:11][0:1:1439][0:1:559]'
                      ,decode_times=False)
scow

In [ ]:
# create an annual mean -- takes a long time to load data
scow_am = scow.mean(dim='time').load()
ds = scow_am.transpose('latitude', 'longitude')

In [ ]:
def quick_quiver(u, v, sampling_x=10, sampling_y=10,
                 scalar=None, mag_max=None, **kwargs):
    x = u.longitude
    y = u.latitude
    slx = slice(None, None, sampling_x)
    sly = slice(None, None, sampling_y)
    sl2d = (sly, slx)
    if scalar is None:
        mag = 0.5*(u**2 + v**2)**0.5
    else:
        mag = scalar

    fig, ax = plt.subplots(**kwargs)
    mag.plot(ax=ax, vmax=mag_max)
    return ax, ax.quiver(x[slx], y[sly], u[sl2d], v[sl2d])

In [ ]:
quick_quiver(ds.scow_zonal_wind_stress, ds.scow_meridional_wind_stress,
             scalar=ds.scow_wind_stress_curl,
             mag_max=2, sampling_x=20,
             figsize=(16,8))
plt.title(r'SCOW Wind Stress (vectors) and Wind Stress Curl (colors: %s)' 
           % scow.scow_wind_stress_curl.attrs['units']);

In [ ]:
omega = 7.29e-5
R = 6.3781e6
f = 2 * omega * np.sin(np.deg2rad(ds.latitude))
beta = 2 * omega / R * np.cos(np.deg2rad(ds.latitude))
rho0 = 1030
ds['V_sv'] = ds.scow_wind_stress_curl / (beta * rho0 * 1e7)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ds.V_sv.plot(vmax=10, ax=ax)
plt.title(r'Sverdrup Transport (m$^2$ s$^{-1}$)')

In [ ]:
ds.coords['dx'] = (R * np.cos(np.deg2rad(ds.latitude))
                   * np.deg2rad(ds.longitude.diff(dim='longitude')))


# this function calculates the cumulative sum of the meridional transport
# where the cumulative sum is taken in the x direction
def plot_sverdrup_streamfunction(ds, levels=20, **kwargs):
    psi = -(ds.V_sv * ds.dx).isel(longitude=slice(None,None,-1)).cumsum(dim='longitude')/1e6
    psi = psi.where(~ds.V_sv.isnull())
    
    fig, ax = plt.subplots(**kwargs)
    psi.plot.contourf(levels=levels)
    return psi

ds_natl = ds.sel(longitude=slice(270, 360), latitude=slice(16, 60)) 
plot_sverdrup_streamfunction(ds_natl, figsize=(16,8))
plt.title('Very Rough Calculation of Sverdrup Transport in the North Atlantic (Sv)')

In [ ]:
ds_npac = ds.sel(longitude=slice(110, 250), latitude=slice(16, 60))
psi_npac = plot_sverdrup_streamfunction(ds_npac, figsize=(16, 8))
plt.title('Very Rough Calculation of Sverdrup Trasport in the North Pacific (Sv)')